In [1]:
import branca
import csv
import datetime
import folium
import sqlite3
import ipywidgets as widgets
import numpy as np
import pandas as pd
import draw_markers
import table_builder


In [2]:
table_builder.create_table("my_db","data/")


In [3]:
connection = sqlite3.connect("my_db")
crsr = connection.cursor()


In [158]:
def build_type_interact(Building_Category):
    
    if Building_Category == "All":
         where_query = "WHERE b.b_id = r.build_id "
    else:
        where_query = str("""WHERE b.b_id = r.build_id and b.b_category = """ + str("'" + Building_Category + "'"))
    select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(r.coll_before) AS col_before, 
    sum(r.coll_during) AS col_during, 
    sum(r.coll_after) AS col_after """
    from_query = """FROM buildings b, Coll_Within_One_Year_of_Const r """
    group_by_query = """ GROUP BY b.b_id, b.b_lat, b.b_long"""
    whole_query = str(select_query + from_query + where_query + group_by_query)
#     print(whole_query)
    crsr.execute(whole_query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['b_latitude'] = temp.b_latitude.astype(float)
    temp['b_longitude'] = temp.b_longitude.astype(float)
    return draw_markers.place_maps(temp)

categories = ['All','COMMERCIAL','MULTIFAMILY', 'INDUSTRIAL','INSTITUTIONAL','SINGLE FAMILY / DUPLEX']


In [4]:
from query_class import CollidiumQuery

# def build_type_interact(building_category):
#     cq = CollidiumQuery
#     cq.set_b_category(building_category)

#     whole_query = cq.get_qstring()
#     print(whole_query)
#     crsr.execute(whole_query)
#     temp = pd.DataFrame(crsr.fetchall())
#     temp.columns = list(map(lambda x: x[0], crsr.description))
#     temp['b_latitude'] = temp.b_latitude.astype(float)
#     temp['b_longitude'] = temp.b_longitude.astype(float)
#     return draw_markers.place_maps(temp)

# categories = ['All','COMMERCIAL','MULTIFAMILY', 'INDUSTRIAL','INSTITUTIONAL','SINGLE FAMILY / DUPLEX']

# build_type_interact('COMMERCIAL')

cq = CollidiumQuery()
cq.set_b_category(['MULTIFAMILY', 'SINGLE FAMILY / DUPLEX'])
# cq.set_b_category('MULTIFAMILY')

In [9]:
def create_map(data, period):
    """
    Creates a map and plots new buildings (location) and collisions (marker size)

    Args:
    data: a dataframe that contains the following fields:
        b_id (int): buiding id number (key)
        b_latitude (float): latitude of building
        b_longitude (float): longitude of building
        col_before (int): Number of collisions that happend in period prior
        to construction
        col_duirng (int): Number of collisions that happend in period during
        construction
        col_after (int): Number of collisions that happend in period after
        construction

    period (string): String of value "before", "during" or "after"
        indicating whether function should plot volume of collisions before,
        during or after construction

    Returns:
    Image of map with building locations plotted on it. The size of the
    marker corresponds to the number of collisions that happened in the
    period (e.g., before, during or after) identified in the inputs

    Raises:
        IndexError: If a dataframe is passed to the create_map function or
        the place_maps function that does not contain the correct fields, an
        Index Error is raised.

        KeyError: If a string other than "before", "during", or "after" is
        passed as the second argument to create_map then a KeyError is
        raised.

        ValueError: If input data is too big it is difficult to render in
        jupyter. If this happens, a ValueError is raised.

    """

    true_set = set(['b_id', 'b_lat', 'b_long', 'before', 'during', 'after'])

    if not set(list(data)) == true_set:
         raise IndexError("Data set does not contain correct fields %s" % true_set)

    if data.shape[0] > 800:
        raise TypeError("Please select 800 or fewer observations from dataframe.")

    location = [np.mean(data['b_lat']), np.mean(data['b_long'])]

    my_map = folium.Map(location=location, zoom_start=11)

    for index, row in data.iterrows(): #pylint: disable=unused-variable
        folium.CircleMarker(
            location=[row['b_lat'], row['b_long']],
            radius=row[str(period)]/20,
            fill=True,
            fill_color='#132b5e').add_to(my_map)
    return my_map


def place_maps(data):
    """
    Draws three maps, plotting buiding permit locations with size
    representing number of collisions. Maps included show collisions,
    before, during and after building construction.

    Args:
    data: a dataframe that contains the following fields:
        b_id (int): buiding id number (key)
        b_latitude (float): latitude of building
        b_longitude (float): longitude of building
        col_before (int): Number of collisions that happend in period prior
        to construction
        col_duirng (int): Number of collisions that happend in period during
        construction
        col_after (int): Number of collisions that happend in period after
        construction.

    Returns:
        Three map images, with building permit locations identified. Maps
        each show building locations (location of dot), and number of
        collisions (size of dot). The three maps shown include collisions
        before construction (left), during construction (middle) and after
        construction (right).

    Raises
        IndexError: If a dataframe is passed to the create_map function or
        the place_maps function that does not contain the correct fields, an
        Index Error is raised.
    """

    map_grid = branca.element.Figure()

    map_1 = create_map(data, "before")
    map_2 = create_map(data, "during")
    map_3 = create_map(data, "after")

    loc_1 = map_grid.add_subplot(1, 3, 1)
    loc_2 = map_grid.add_subplot(1, 3, 2)
    loc_3 = map_grid.add_subplot(1, 3, 3)

    loc_1.add_child(map_1)
    loc_2.add_child(map_2)
    loc_3.add_child(map_3)

    return map_grid


In [25]:
def generate_table(query):
    connection = sqlite3.connect("my_db")
    crsr = connection.cursor()
    crsr.execute(query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['b_lat'] = temp.b_lat.astype(float)
    temp['b_long'] = temp.b_long.astype(float)
    return temp

def build_type_interact(building_type):
    cq = CollidiumQuery()
    cq.set_b_category(building_category)
    whole_query = cq.get_qstring()

    mapping_data = generate_table(whole_query)
    return draw_markers.place_maps(mapping_data)
categories = ['All','COMMERCIAL','MULTIFAMILY', 'INDUSTRIAL','INSTITUTIONAL','SINGLE FAMILY / DUPLEX']

In [26]:
widgets.interact(build_type_interact, building_category = categories, data_base = "my_db")



interactive(children=(Dropdown(description='building_category', options=('All', 'COMMERCIAL', 'MULTIFAMILY', '…

NameError: name 'test' is not defined

In [139]:
def time_frame_interact(num_days):

    where_query = str("WHERE b.b_id = r.build_id AND (r.coll_days_from_build BETWEEN 0 AND " + 
                      str(num_days) + " OR r.coll_days_from_build BETWEEN -1 AND" + str(-num_days) + ")")
    select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(r.coll_before) AS col_before, 
    sum(r.coll_during) AS col_during, 
    sum(r.coll_after) AS col_after """
    from_query = """FROM buildings b, Coll_Within_One_Year_of_Const r """
    group_by_query = """ GROUP BY b.b_id"""
    whole_query = str(select_query + from_query + where_query + group_by_query)
    crsr.execute(whole_query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['col_during'] = temp['col_during'] * num_days/365
    temp['b_latitude'] = temp.b_latitude.astype(float)
    temp['b_longitude'] = temp.b_longitude.astype(float)
#     print("bef: ", sum(temp.col_before), "dur: ",sum(temp.col_during), "aft: ",sum(temp.col_after))
    return draw_markers.place_maps(temp)

In [141]:
widgets.interact(time_frame_interact, num_days = (65,365,60))

interactive(children=(IntSlider(value=185, description='num_days', max=365, min=65, step=60), Output()), _dom_…

<function __main__.time_frame_interact(num_days)>

In [125]:
def build_value_interact(value):

    where_query = str("""WHERE b.b_id = r.build_id and (b.b_value > """ + str(value[0]*1000000) + ")") # """ AND b.b_value < """ + str(value[1]*1000000) + ")")
    select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(coll_before) AS col_before, 
    sum(coll_during) AS col_during, 
    sum(coll_after) AS col_after """
    from_query = """FROM buildings b, Coll_Within_One_Year_of_Const r """
    group_by_query = """ GROUP BY b.b_id, b.b_lat, b.b_long"""
    whole_query = str(select_query + from_query + where_query + group_by_query)
#     print(whole_query)
    crsr.execute(whole_query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['b_latitude'] = temp.b_latitude.astype(float)
    temp['b_longitude'] = temp.b_longitude.astype(float)
    return draw_markers.place_maps(temp)



In [134]:
value = [100,10]

# where_query = str("""WHERE b.b_id = r.build_id and (b.b_value > """ + str(value[0]*1000000) + ")") # """ AND b.b_value < """ + str(value[1]*1000000) + ")")
where_query = str("""WHERE b.b_value < 1500000""")# + str(value[0]*1000000)) #+ """ AND b.b_value < """ + str(value[1]*1000000) + ")")

select_query = """SELECT b.b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(r.coll_before) AS col_before, 
    sum(r.coll_during) AS col_during, 
    sum(r.coll_after) AS col_after,
    min(b.b_value) """
from_query = """FROM buildings b INNER JOIN Coll_Within_One_Year_of_Const r ON b.b_id = r.build_id """
group_by_query = """ GROUP BY b.b_id, b.b_lat, b.b_long"""
whole_query = str(select_query + from_query + where_query + group_by_query)
print(whole_query)
crsr.execute(whole_query)
temp = pd.DataFrame(crsr.fetchall())
temp.columns = list(map(lambda x: x[0], crsr.description))
temp
#     temp['b_latitude'] = temp.b_latitude.astype(float)
#     temp['b_longitude'] = temp.b_longitude.astype(float)
#     return draw_markers.place_maps(temp)



SELECT b.b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(r.coll_before) AS col_before, 
    sum(r.coll_during) AS col_during, 
    sum(r.coll_after) AS col_after,
    min(b.b_value) FROM buildings b INNER JOIN Coll_Within_One_Year_of_Const r ON b.b_id = r.build_id WHERE b.b_value < 1500000 GROUP BY b.b_id, b.b_lat, b.b_long


,b_id,b_latitude,b_longitude,col_before,col_during,col_after,min(b.b_value)
0,6153914,47.55680616,-122.3864579,11,31.383178,43,12859426
1,6180940,47.61582105,-122.3461666,319,313.279352,283,14982063
2,6226141,47.64464664,-122.3261037,37,44.031746,32,10618568
3,6268969,47.62490137,-122.3655399,60,0.000000,49,13627890
4,6281378,47.67054426,-122.3880165,47,55.020188,56,11750000
5,6282461,47.56965147,-122.2901043,57,47.007576,47,1099065
6,6284321,47.61698493,-122.3319657,255,238.671199,50,142981553
7,6285721,47.60911773,-122.3383204,435,365.927573,0,103861514
8,6294017,47.64866803,-122.3839158,26,23.347548,24,1005000
9,6295045,47.66414599,-122.3166552,55,166.877432,161,12567914


# Everything below here is work in progress

In [ ]:
def filter_collision_victims(victims):
    if victims = "Cyclist":
        where_clause = "WHERE c_cyc = 1 "
    if victims = "Pedenstrian":
        where_clause = "WHERE c_ped = 1 "
    if victims = "Pedestrian or Cyclist":
        where_clause
    
    select_query = ""

In [142]:
def col_ped_bike_interact(victims):

    where_query = str("""WHERE b.b_id = r.build_id and (b.b_value > """ + str(min_value) + ")") # """ AND b.b_value < """ + str(max_value) + ")")
    select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(coll_before) AS col_before, 
    sum(coll_during) AS col_during, 
    sum(coll_after) AS col_after """
    from_query = """FROM buildings b, Collision_count_BDA_Const_all r """
    group_by_query = """ GROUP BY b.b_id"""
    whole_query = str(select_query + from_query + where_query + group_by_query)
    print(whole_query)
    crsr.execute(whole_query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['b_latitude'] = temp.b_latitude.astype(float)
    temp['b_longitude'] = temp.b_longitude.astype(float)
    return draw_markers.place_maps(temp)
    

In [101]:
def my_func(value):
    value = value
    return value[0], value[1]

widgets.interact(my_func, value=widgets.IntRangeSlider(value=[0, 300], min=0, max=300, step=10,
                                                           description='boo',
                                                           continuous_update=False
                                                           )
                              )

interactive(children=(IntRangeSlider(value=(0, 300), continuous_update=False, description='boo', max=300, step…

<function __main__.my_func(value)>